In [ ]:
import pandas as pd, numpy as np, tensorflow as tf
from sklearn.model_selection import train_test_split
from sklearn.preprocessing  import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
import joblib, os

In [ ]:
# 0. Reprodutibilidade
np.random.seed(42); tf.random.set_seed(42)

# 1. Dados -------------------------------------------------------------
df = pd.read_csv("rain_risk_training_data.csv")      # mesmas colunas
X  = df["precipitation_mm"].values.reshape(-1, 1).astype("float32")
y  = df["risk_code"]        .values.astype("int32")   # 0–4

# Escalonar (guarda média e desvio p/ ESP)
scaler = StandardScaler().fit(X)
X_scaled = scaler.transform(X)

X_tr, X_te, y_tr, y_te = train_test_split(
    X_scaled, y, test_size=0.2, stratify=y, random_state=42)

# 2. Rede – 1 ➜ 16 ➜ 8 ➜ 5 --------------------------------------------
model = Sequential([
    Input(shape=(1,)),
    Dense(32, activation="relu"),
    Dense(16 , activation="relu"),
    Dense(8 , activation="softmax")   # 5 classes: 0..4
])
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])

# 3. Treino
es = EarlyStopping(monitor='val_loss', patience=15, restore_best_weights=True)
rl = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=7, min_lr=1e-4)
hist = model.fit(X_tr, y_tr,
                 validation_data=(X_te, y_te),
                 epochs=120, batch_size=32,
                 callbacks=[es, rl], verbose=1)

print("Acurácia teste:", model.evaluate(X_te, y_te, verbose=0)[1])

# 4. Salvar ------------------------------------------------------------
model.save("risk_model.h5")
joblib.dump(scaler, "scaler.pkl")

# 5. Converter p/ TFLite (INT8) ---------------------------------------
conv = tf.lite.TFLiteConverter.from_keras_model(model)
conv.optimizations = [tf.lite.Optimize.DEFAULT]
tflm = conv.convert()
open("risk_model.tflite", "wb").write(tflm)

# 6. Mostrar tamanhos
print("H5: %.1f KB  |  TFLite: %.1f KB"
      % (os.path.getsize("risk_model.h5")/1024,
         os.path.getsize("risk_model.tflite")/1024))

# 7. Exportar scaler p/ copiar depois
print("mean =", float(scaler.mean_[0]),
      "  std =", float(scaler.scale_[0]))

Epoch 1/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 1s 61ms/step - accuracy: 0.2064 - loss: 2.0898 - val_accuracy: 0.2000 - val_loss: 2.0661 - learning_rate: 0.0010
Epoch 2/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.2064 - loss: 2.0591 - val_accuracy: 0.2000 - val_loss: 2.0363 - learning_rate: 0.0010
Epoch 3/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 17ms/step - accuracy: 0.2064 - loss: 2.0300 - val_accuracy: 0.2000 - val_loss: 2.0076 - learning_rate: 0.0010
Epoch 4/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step - accuracy: 0.3003 - loss: 2.0017 - val_accuracy: 0.4500 - val_loss: 1.9790 - learning_rate: 0.0010
Epoch 5/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4614 - loss: 1.9734 - val_accuracy: 0.4000 - val_loss: 1.9501 - learning_rate: 0.0010
Epoch 6/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4614 - loss: 1.9446 - val_accuracy: 0.4000 - val_loss: 1.9203 - learning_rate: 0.0010
Epoch 7/120
5/5 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.4839 - loss: 1.9146 - val_accuracy: 

Acurácia teste: 0.949999988079071
Saved artifact at '/tmp/tmp67nxe13p'. The following endpoints are available:

* Endpoint 'serve'
  args_0 (POSITIONAL_ONLY): TensorSpec(shape=(None, 1), dtype=tf.float32, name='keras_tensor_36')
Output Type:
  TensorSpec(shape=(None, 8), dtype=tf.float32, name=None)
Captures:
  135451043880720: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135451195693712: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135451195688528: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135451195694096: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135451195695248: TensorSpec(shape=(), dtype=tf.resource, name=None)
  135451136604112: TensorSpec(shape=(), dtype=tf.resource, name=None)
H5: 36.0 KB  |  TFLite: 4.9 KB
mean = 14.20085000211373   std = 12.72817877445062
